In [90]:
import pandas as pd
import pickle
import numpy as np

In [91]:
# Load IDs
with open('uniqueIDs.pkl', 'rb') as f:
    uniqueIDs = pickle.load(f)

In [92]:
all_users_folder = 'data/all_users/'

# Winning model = model 8

In [93]:
model_folder = all_users_folder + 'mod8/'
data_mod = pd.read_pickle(model_folder+'mod_param_fits.pkl')
mod_info = pd.read_pickle(model_folder+'mod_parameters.pkl')

# Display stats
data_mod_num = data_mod.apply(pd.to_numeric)
data_mod_num.drop('ID', axis=1).describe()

# Load data
ev_per_trial = pd.read_pickle(all_users_folder + 'mod8/mod_ev_per_trial.pkl');
p_hit_per_trial = pd.read_pickle(all_users_folder + 'mod8/mod_p_hit_per_trial.pkl');
trialsNos = pd.read_pickle(all_users_folder + 'mod8/trialsNos.pkl');
PEs = pd.read_pickle(all_users_folder + 'mod8/PEs.pkl');
fbs = pd.read_pickle(all_users_folder + 'mod8/fbs.pkl');
hits = pd.read_pickle(all_users_folder + 'mod8/hits.pkl');
shrink_PIs = pd.read_pickle(all_users_folder + 'mod8/shrink_PIs.pkl');
shrink_alphas = pd.read_pickle(all_users_folder + 'mod8/shrink_alphas.pkl');

In [97]:
data_mod_num.to_excel(all_users_folder + 'excel_summary/model_param_summary.xlsx')

In [95]:
for ID in uniqueIDs:
    
    # Extract for ID
    ev_singID = ev_per_trial[ev_per_trial['ID']==ID].drop('ID', axis=1, inplace=False).set_index('Cue',inplace=False)
    pe_singID = PEs[PEs['ID']==ID].drop('ID', axis=1, inplace=False).set_index('Cue',inplace=False)
    phit_singID = p_hit_per_trial[p_hit_per_trial['ID']==ID].drop('ID', axis=1, inplace=False).set_index('Cue',inplace=False)
    trials_singID = trialsNos[trialsNos['ID']==ID].drop('ID', axis=1, inplace=False).set_index('Cue',inplace=False)
    fbs_singID = fbs[fbs['ID']==ID].drop('ID', axis=1, inplace=False).set_index('Cue',inplace=False)
    hits_singID = hits[hits['ID']==ID].drop('ID', axis=1, inplace=False).set_index('Cue',inplace=False)
    shrink_PIs_singID = shrink_PIs[shrink_PIs['ID']==ID].drop('ID', axis=1, inplace=False).set_index('Cue',inplace=False)
    shrink_alphas_singID = shrink_alphas[shrink_alphas['ID']==ID].drop('ID', axis=1, inplace=False).set_index('Cue',inplace=False)
    
    # Shifts trials PEs (starts on trial 2)
    pe_singID_shifted = pe_singID.drop(28, axis=1, inplace=False)
    pe_singID_shifted.columns = [col+1 for col in pe_singID_shifted.columns]
    pe_singID_shifted.insert(0, 1, np.nan)

    # Shifts trials shrinking alphas (starts on trial 2)
    shrink_alphas_shifted = shrink_alphas_singID.drop(28, axis=1, inplace=False)
    shrink_alphas_shifted.columns = [col+1 for col in shrink_alphas_shifted.columns]
    shrink_alphas_shifted.insert(0, 1, np.nan)
    
    cue_ascending = []
    trials_ascending = []
    evs_ascending = []
    pes_ascending = []
    phits_ascending = []
    fbs_ascending = []
    hits_ascending = []
    cue_trials_ascending = []
    shrink_pis_ascending = []
    shrink_alphas_ascending = []

    Ntrials = 112

    for trial in range(1,Ntrials+1):

        # keep only value on specific trial
        ev_on_trial = ev_singID[trials_singID==trial]
        pe_on_trial = pe_singID_shifted[trials_singID==trial]
        shrink_alphas_on_trial = shrink_alphas_shifted[trials_singID==trial]
        shrink_pis_on_trial = shrink_PIs_singID[trials_singID==trial]
        phit_on_trial = phit_singID[trials_singID==trial]
        fb_on_trial = fbs_singID[trials_singID==trial]
        hits_on_trial = hits_singID[trials_singID==trial]

        # drop everything not interesting
        cue_ev_on_trial = ev_on_trial.dropna(axis=0, how='all').dropna(axis=1)
        cue_pe_on_trial = pe_on_trial.dropna(axis=0, how='all').dropna(axis=1)
        cue_phit_on_trial = phit_on_trial.dropna(axis=0, how='all').dropna(axis=1)
        cue_fb_on_trial = fb_on_trial.dropna(axis=0, how='all').dropna(axis=1)
        cue_hits_on_trial = hits_on_trial.dropna(axis=0, how='all').dropna(axis=1)
        cue_shrink_alpha_on_trial = shrink_alphas_on_trial.dropna(axis=0, how='all').dropna(axis=1)
        cue_shrink_pi_on_trial = shrink_pis_on_trial.dropna(axis=0, how='all').dropna(axis=1)

        # store cue
        cue_ascending.append(cue_ev_on_trial.index[0])

        # store trial
        trials_ascending.append(trial)

        # store cue_trial
        cue_trial = cue_ev_on_trial.columns[0]
        cue_trials_ascending.append(cue_trial)

        # store EV, PE, phit, fb, hit
        evs_ascending.append(cue_ev_on_trial.values[0][0])
        phits_ascending.append(cue_phit_on_trial.values[0][0])
        fbs_ascending.append(cue_fb_on_trial.values[0][0])
        hits_ascending.append(cue_hits_on_trial.values[0][0])
        shrink_pis_ascending.append(cue_shrink_pi_on_trial.values[0][0])

        if cue_trial == 1:
            pes_ascending.append(np.nan)
            shrink_alphas_ascending.append(np.nan)
        else:
            shrink_alphas_ascending.append(cue_shrink_alpha_on_trial.values[0][0])
            pes_ascending.append(cue_pe_on_trial.values[0][0])


    data_per_trial = pd.DataFrame(list(zip(trials_ascending, cue_ascending, cue_trials_ascending, 
                                 pes_ascending, shrink_alphas_ascending,
                                 evs_ascending, shrink_pis_ascending, phits_ascending, 
                                 hits_ascending, fbs_ascending)), 
                        columns=['Trial', 'Cue', 'CueTrial', 
                                 'mod_PE', 'mod_learning_rate',
                                 'mod_EV', 'mod_bias_to_hit', 'mod_hit_probability', 
                                 'Hit', 'FB'])
    
    data_per_trial.to_excel(all_users_folder + 'excel_summary/userID_' + ID + '.xlsx')
    